# INFO F422 - Statistical Fundation of Machine Learning
## Project "House Prices : Advanced Regression Techniques"

    Erica Berghman
    Master 1 - Brussels Engineer School

## Abstract

## 1. Introduction 

* with dataset description, goals, and an overview of the report structure 

Starting from a data set with 81 criteria about houses and their selling price, the goal is to create a model capable of predicting the price of other houses given some of these criterias. A good model description is a model that has been refined multiple types. This report will show the methodology used to construct a model for this particular problem. It is based on the methodology of the Chapter 6 of the syllabus.

## 2. Feature selection 
Methodology and main results

The text must contain the list of selected variables and the motivation of their choice. The use of formulas, tables and pseudo-code to describe the feature selection procedure is encouraged. 

In [298]:
dataSample = 400
mean = T          # variable to determine if we use the mean or the median to replace the NA values
set.seed(2)

source("replaceNA.R")

### 2.1 Preprocessing the data set

In order to get a model, the data must be preprocessed. Firstly we read the data given and we take a sample set of 400 houses out of the 1460. There is 81 criteria.

In [299]:
data<-read.csv("input/train.csv")
data.sample<-data[sample(nrow(data),dataSample),]
#dim(data.sample)
#data[1:2,]

The categorical (factor) criterias are removed.

In [300]:
factor_variables<-which(sapply(data.sample[1,],class)=="factor")
data.sample.nofactor<-data.sample[,-factor_variables]
data.sample.factor<-data.sample[,factor_variables]
#summary(data.sample.factor)

library(dummies)
variable_to_keep<-c("CentralAir", "Street", "LotShape")
data_factor_onehot <- dummy.data.frame(data.sample.factor[,variable_to_keep], sep="_")
data.nofactor.extended<-cbind(data.sample.nofactor,data_factor_onehot)


#### Missing data 
The missing values (NA) are replaced by an estimator of these values (eg. mean or median).Ca

In [301]:
if (mean) {
    data_preprocessed<-data.frame(apply(data.nofactor.extended,2,replace_na_with_mean_value)) 
} else {
    data_preprocessed<-data.frame(apply(data.nofactor.extended,2,replace_na_with_median_value))
}

### 2.2 Feature selection methodology

#### Redundant and irrelevant features 

The "Id" column which is irrelevant is deleted.

In [302]:
data_preprocessed<-data_preprocessed[,setdiff(colnames(data_preprocessed),"Id")]

The criterias that are redundant (linear combination of others criterias and correlation > 0.95) are deleted.

In [303]:
library(caret)
findLinearCombos(data_preprocessed)
linearCombo.idx <- findLinearCombos(data_preprocessed)$remove
if (!is.null(linearCombo.idx)) data_preprocessed<-data_preprocessed[,-linearCombo.idx]

correlation.matrix <- cor(data_preprocessed)
correlation.matrix[upper.tri(correlation.matrix)] <- 0
diag(correlation.matrix) <- 0
data.uncorrelated <- data_preprocessed[,!apply(correlation.matrix,2,function(x) any(abs(x) > 0.95))]

$linearCombos
$linearCombos[[1]]
[1] 12  9 10 11

$linearCombos[[2]]
[1] 16 13 14 15

$linearCombos[[3]]
[1] 41 38 39 40

$linearCombos[[4]]
[1] 45 38 39 42 43 44


$remove
[1] 12 16 41 45

In [297]:
X<-data.uncorrelated[,setdiff(colnames(data.uncorrelated),"SalePrice")]
Y<-data.uncorrelated[,"SalePrice"]
X.scale <- data.frame(scale(X))
Y.scale <- scale(Y)

N<-nrow(X)
n<-ncol(X)


Three feature selection methods are implemented in the featureSelection file:

**1. Filter method using correlation with the variable to determine.**

   It create a subset of features, removing from the whole features set the ones less likely to determine the variable (SalePrice). It is robust to overfitting and effective in computational time. However it might select redundant variables as the interraction between the variables is not taken in consideration.  
   
**2. Wrapper method**

   Its a cyclic method where a subset of variable is created and evaluated by the Learning Algorithm, modifying the chosen subset. This is done until the best subset is generated.  
    
**3. Embedded method**

In [187]:
source("featureSelection.R")
featureSelection.method = "correlation"

## Model selection  
Methodology and main results

For the learning method, the only packages that may be used are those seen during the exercise classes : stats, nnet, tree, lazy, and e1071, for linear models, neural networks, decision trees, nearest neighbours and SVM, respectively.

The accuracy of the regression models during the selection process should be assessed by using the root mean squared error between the logarithm of the
predicted value and the logarithm of the observed sale price.

The text must mention the different
(and at least three) models which have been taken into consideration and the procedure used for model assessment and selection. The use of formulas,
tables and pseudo-code to describe the feature selection procedure is encouraged

## Ensemble techniques : Combination of models strategy
Methodology and main results

The text should mention the different models taken into consideration as well as the techniques used for the combination.

## Discussion and conclusion: 
Summary of your work, and discussion of what worked well, not well, why, what insights you got from the analyses you made. 